# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,942.355416,0.170856,1234,942.526272,942.184560,1885.052543,1884.369120
6,334.701362,0.197143,1234,334.898505,334.504218,669.797010,669.008436
7,652.170757,0.894173,1234,653.064930,651.276585,1306.129860,1302.553170
8,391.615040,0.006623,1234,391.621663,391.608416,783.243326,783.216832
9,499.906555,0.417660,1234,500.324215,499.488895,1000.648430,998.977789
10,297.766636,0.587783,1234,298.354418,297.178853,596.708836,594.357706
11,271.759592,-0.629808,1234,271.129785,272.389400,542.259569,544.778801
12,582.826181,-0.989908,1234,581.836273,583.816090,1163.672545,1167.632179
13,819.638269,0.440147,1234,820.078416,819.198122,1640.156832,1638.396243
14,746.649002,0.306690,1234,746.955692,746.342312,1493.911384,1492.684624


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-942.526272,942.184560
6,-334.898505,334.504218
7,-653.064930,651.276585
8,-391.621663,391.608416
9,-500.324215,499.488895
10,-298.354418,297.178853
11,-271.129785,272.389400
12,-581.836273,583.816090
13,-820.078416,819.198122
14,-746.955692,746.342312


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-942.526272,942.184560
6,-334.898505,334.504218
7,-653.064930,651.276585
8,-391.621663,391.608416
9,-500.324215,499.488895
10,-298.354418,297.178853
11,-271.129785,272.389400
12,-581.836273,583.816090
13,-820.078416,819.198122
14,-746.955692,746.342312


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-942.5262716 ,  942.18455985],
       [-334.89850499,  334.5042182 ],
       [-653.0649302 ,  651.2765848 ],
       [-391.6216631 ,  391.60841619],
       [-500.32421507,  499.48889467],
       [-298.3544181 ,  297.17885305],
       [-271.12978451,  272.38940043],
       [-581.8362726 ,  583.81608951],
       [-820.07841608,  819.1981217 ],
       [-746.95569181,  746.34231202]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1883.052543,-1884.369120
6,667.797010,-669.008436
7,1304.129860,-1302.553170
8,781.243326,-783.216832
9,998.648430,-998.977789
10,594.708836,-594.357706
11,540.259569,-544.778801
12,1161.672545,-1167.632179
13,1638.156832,-1638.396243
14,1491.911384,-1492.684624


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
